# Demo for specXplore importing & session initiation

This demo jupyter notebooks showcases how to use the specXplore importing pipeline and how to start up the interactive exploration dashboard.
To run this notebook you need to set up the following:

1. You need to have **installed specXplore** using the [instructions on github](https://github.com/kevinmildau/specXplore) and open this notebook as indicated.
2. You need to have **downloaded the ms2query model and library files** from the corresponding zenodo repositories for [positive](https://zenodo.org/records/10527997) or [negative](https://zenodo.org/records/10528030) mode. Please save each mode's model files in a different folder, e.g. "models_positive" & "models_negative" to avoid errors in the specXplore pipeline regarding model loading. Both modes require separate runs as specXplore is ion-mode specific. *For model customization please refer to the respective package documentation and repositories*.

In addition, you need the **demo.mgf** file and its filepath to run the demo. You can also replace the demo.mgf with your own .mgf file data provided the .mgf file entries have the required feature_id and precursor_mz/pepmass entries. The following is an example entry you may see in your .mgf file when opening it in a text editor:

```text 
BEGIN IONS
FEATURE_ID=1961
PEPMASS=105.03386
SCANS=1961
RTINSECONDS=746.836
CHARGE=1+
MSLEVEL=2
51.02379 5.7E4
53.03930 4.1E5
70.08131 2.5E4
END IONS
```

Note that there is a strict requirement for **"feature_id"** to be present and unique for each spectrum. If this is not the case, the spectral data will have to be processed with [matchms](https://matchms.readthedocs.io/en/latest/) ([see example](https://github.com/kevinmildau/specXplore/blob/master/notebooks/example-feature-id-processing.ipynb)) or other means. In the above example, the original feature identifying columns was SCANS. A copy of SCANS was added to each entry with key FEATURE_ID via matchms. Searching for all exact matches of "SCANS=" and replacing them with "FEATURE_ID=" using any text-editor would be a quick solution.

Note that while specXplore runs with any LC-MS/MS spectral data provided in .mgf file format, it is highly recommended to perform data processing beforehand to reduce the number of features and increase their quality for exploration and annotation purposes. For more information on processing LC-MS/MS data please refer to your vendor specific software manuals or open source processing tools such as [MZmine3](https://mzmine.github.io/mzmine_documentation/getting_started.html). Alternative file formats may be imported and converted using [matchms.importing](https://matchms.readthedocs.io/en/latest/api/matchms.importing.html) and [matchms.exporting](https://matchms.readthedocs.io/en/latest/api/matchms.exporting.html) for compatibility with specXplore.

# Module Loading

The code cell below loads the models required for specXplore and this demo-notebook to run. These should be available if you followed the set-up and notebook starup instructions.

In [ ]:
import specxplore.importing
from specxplore.session_data import load_specxplore_object_from_pickle
from specxplore.dashboard import SpecxploreDashboard
import os # for filepath generality across operating systems of this notebook; the user may use operating specific filepaths

# User Input

The following line of code requires the user to provide data and model filepaths. 
For the demo, the data is demo_data.mgf which is assumed to be stored inside a folder called data. 
The models are assumed to be stored in a folder called "models_positive".
Both the file and the directory are assumed to be inside the same path that jupyter notebook is in, e.g. the folder contents are:

```
|--[parent folder]
     |-- demo.ipynb                <-- this demo notebook
     |-- models_positive           <-- the folder with downloaded model files, this directory is a required input
          |-- ...                  <-- model & library files from ms2query for positive mode
     |-- data                      <-- the folder with the demo data
          |-- demo_data.mgf        <-- the demo data in mgf file format, this filepath is a required input
     |-- output                    <-- folder to place output data into, unless specified 
                                       otherwise. Folder must exist.
```

When run with default settings, the specXplore pipeline will create a folder with name "output" within which ms2query results and the specXplore session data will be stored.

To run the demo, place the required folders and files as indicated or change the relative filepaths to correspond to stored file locations.

In [ ]:
data_file_path = os.path.join("data", "demo_data.mgf") # USER INPUT. MGF FILEPATH. BOTH RELATIVE AND ABSOLUTE PATHS WILL WORK
model_file_path = "models_positive" # USER INPUT. MODEL DIRECTORY. BOTH RELATIVE AND ABSOLUTE PATHS WILL WORK

specxplore_output_filepath_ms2query = os.path.join("output", "ms2query_output.csv") # default, can be modifed
specxplore_output_filepath_session = os.path.join("output", "specxplore_session_data.pickle") # default, can be modifed

# specXplore importing pipeline

Once the filepaths are defined the specXplore importing pipeline runs nearly automatically. To run specXplore, run the three code blocks below. These will instantiate the pipeline, attach the data, run processing, and produce output results. A tuning grid for t-SNE embedding and k-medoid clustering is run. The pipeline below is scripted such that values are automatically selected. However, the user should inspect the tabular output tables provided by specXplore to assess which values of k and which values of perplexity to make use of. 

For selecting k-medoids clustering values, higher silhouette scores form an indicator of better clustering performence. 
For t-SNE, the highest distance correlation metric is usually a good indicator of embedding performance.
The user can select the parameters to use by setting ```selected_tsne_iloc``` to a suitable integer locations (iloc) from the output table (e.g. ```selected_tsne_iloc = 0``` or ```selected_tsne_iloc = 2```), and set ```selected_kmedoid_ilocs``` to a suitable list of integer locations (e.g. ``` selected_kmedoid_ilocs = [0, 3, 4]```).

In addition, the user may provide a list of feature_id identifiers corresponding to the features to be highlighted as in-silico spike in standards. These could be imported from a list of known entries (e.g. from reading in excel data using [pandas](https://pandas.pydata.org/docs/user_guide/10min.html) and [exporting to list](https://pandas.pydata.org/docs/reference/api/pandas.Series.to_list.html)).

Run the following two cell to perform all pipeline steps with default settings. Note that similarity computations and ms2query may take a long time for large datasets.

In [ ]:
pipeline_instance = specxplore.importing.specxploreImportingPipeline()
pipeline_instance.attach_spectra_from_file(data_file_path)
pipeline_instance.run_spectral_processing()
pipeline_instance.run_spectral_similarity_computations(model_file_path)
pipeline_instance.run_ms2query(model_file_path, results_filepath = specxplore_output_filepath_ms2query)
pipeline_instance.run_and_attach_tsne_grid(perplexity_values = [5,10,15,100]) # Consider changing perplexity values input
pipeline_instance.run_and_attach_kmedoid_grid(k_values = [5,10,15,100]) # Consider changing k values input

In [ ]:
# USER INPUT ; DEFAULTS ARE UNLIKELY TO MAKE SENSE FOR USER DATA!
features_to_highlight = ['1961', '76', '198'] # provide feature ids to highlight as string or empty list []
selected_tsne_iloc = 1 # select t-SNE settings using iloc from tabular output of previous cell
selected_kmedoid_ilocs = [0, 2] # select k-values settings using iloc from tabular output of previous cell
# Complete pipeline run
pipeline_instance.attach_feature_highlights(features_to_highlight)
pipeline_instance.select_tsne_settings(selected_tsne_iloc)
pipeline_instance.select_kmedoid_settings(selected_kmedoid_ilocs)
pipeline_instance.export_specxplore_session_data(force = True, filepath = specxplore_output_filepath_session)

# Run specXplore dashboard session
The following two code blocks are exchangeable and have as their effect to run the specXplore dashboard with the data generated in the previous code blocks. The specXplore session data is loaded from the default filepath relative to the current filepath, a dashboard is instantiated, and the interactive session is started. The first block opens the dashboard within Jupyter, while the second generated a local server to interact with.

In [ ]:
data = load_specxplore_object_from_pickle(filepath=specxplore_output_filepath_session)
dashboard = SpecxploreDashboard(data)
dashboard.run_app()

In [ ]:
data = load_specxplore_object_from_pickle(filepath=specxplore_output_filepath_session)
dashboard = SpecxploreDashboard(data)
dashboard.run_app(jupyter_mode = "external")

To close the dashboard from running on the port, restart the jupyter kernel. Note that the specxplore session data is saved to file and can be reloaded here without rerunning the module. To do so, rerun the first code line importing the modules, as well as the second code block defining the filepaths.

# Appendix

When running specXplore from within the Jupyter notebook and re-running a line of code you may notice that certain steps which take a longer time to run block re-running if output files already exist. For instance, if the ms2query results .csv file is already available, pipeline.run_ms2query() will fail to run unless the force setting is set to True, prompting overwriting of the previous ms2query results.

To make use of existing results for ms2query, use the following as a substitute to run_ms2query():

```python
pipeline_instance.attach_ms2query_results(specxplore_output_filepath_ms2query)
```

Here, if  ```specxplore_output_filepath_ms2query``` is not provided and ```pipeline_instance.attach_ms2query_results()``` is run instead, specXplore will look automatically at the default location of output -> ms2query_output.csv. This will block execution if files are not available.
